In [ ]:
import asyncio
from polymarket.src.stream.binance import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket.src.db.ingest_stream import insert_into_db, parse_message

conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;')
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()


async def main(conf, queue, symbols, table_name):
    executor = ProcessPoolExecutor(max_workers=10)
    db_task = asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor))
    await binance_price_ws(queue=queue, symbols=symbols)
    await queue.put(None)
    await db_task

await main(conf, queue, symbols, table_name)

In [6]:
import asyncio
from polymarket.src.stream.binance import binance_price_ws
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from polymarket.src.db.ingest_stream import insert_into_db, parse_message
conf = (
    'http::addr=localhost:9000;'
    'username=admin;password=quest;'
    'auto_flush_rows=100;auto_flush_interval=1000;')
table_name = "trade_stream"
symbols = ["btcusdt"]
queue = asyncio.Queue()

async def main():

    executor = ThreadPoolExecutor(max_workers=10)
    producers = [asyncio.create_task(binance_price_ws(queue=queue, symbols=symbols)),]
    workers = [asyncio.create_task(insert_into_db(conf, table_name, queue, parse_message, executor))]

    await asyncio.gather(*producers, *workers)

await main()

Starting DB insert task...
Connected to Binance Futures Ticker WebSocket!
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768251402493, 's': 'BTCUSDT', 'p': '788.60', 'P': '0.870', 'w': '91325.23', 'c': '91419.30', 'Q': '0.001', 'o': '90630.70', 'h': '92488.00', 'l': '90066.00', 'v': '169413.096', 'q': '15471689642.23', 'O': 1768164960000, 'C': 1768251402491, 'F': 7091872975, 'L': 7095689053, 'n': 3807084})
{'symbol': 'BTCUSDT', 'open': 90630.7, 'volume': 169413.096, 'timestamp_binance': 1768251402491}
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768251404656, 's': 'BTCUSDT', 'p': '788.70', 'P': '0.870', 'w': '91325.23', 'c': '91419.40', 'Q': '0.007', 'o': '90630.70', 'h': '92488.00', 'l': '90066.00', 'v': '169413.190', 'q': '15471698235.66', 'O': 1768164960000, 'C': 1768251404654, 'F': 7091872975, 'L': 7095689057, 'n': 3807088})
{'symbol': 'BTCUSDT', 'open': 90630.7, 'volume': 169413.19, 'timestamp_binance': 1768251404654}
('btcusdt@ticker', {'e': '24hrTicker', 'E': 1768251406288, 's': '

CancelledError: 